In [11]:
import glob
import pickle
from tqdm import tqdm

from sqlalchemy import Column, Integer, String, Date, Text, ARRAY, FLOAT

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

from db_utils import WapoQueries, WapoDocEmbeddings, gen_session, refresh_db, get_wapo_doc2query, get_wapo_emb

import ir_datasets
import zipfile

In [2]:
#! createdb -h postgres datasets

In [ ]:
server_address = 'postgresql://root@postgres:5432/'

In [ ]:
session = gen_session(server_address=server_address, db_name="datasets")

In [ ]:
Base = declarative_base()

In [ ]:
#define tables
class WapoEntry(Base):
    __tablename__ = 'wapo'
    doc_id = Column(String, primary_key=True)
    url = Column(String)
    title = Column(String)
    author = Column(String)
    kicker = Column(Text)
    body = Column(Text)
    
    def __repr__(self):
        repr_str = f"doc_id={self.doc_id}, url={self.url}, title={self.title}, author={self.author},"\
        f"kicker={self.kicker}, body={self.body}"
        
        return repr_str
    
class NytEntry(Base):
    __tablename__ = 'nyt'
    doc_id = Column(String, primary_key=True)
    headline = Column(String)
    body = Column(String)
    
    def __repr__(self):
        repr_str = f"doc_id={self.doc_id}, headline={self.headline}, body={self.body}"

        return repr_str

In [ ]:
#create tables

engine = create_engine(server_address + "datasets")
Base.metadata.create_all(engine)

In [25]:
dataset_nyt = ir_datasets.load("nyt")
dataset_wapo = ir_datasets.load("wapo/v2")

In [17]:
with zipfile.ZipFile("/workspace/data/nyt/nyt.tgz.pklz4.zip","r") as zip_ref:
      zip_ref.extractall("/root/.ir_datasets/nyt/")

In [21]:
with zipfile.ZipFile("/workspace/data/wapo/WashingtonPost.v2.tar.gz.pklz4.zip","r") as zip_ref:
      zip_ref.extractall("/root/.ir_datasets/wapo/")

In [27]:
docstore_nyt = dataset_nyt.docs_store()
docstore_wapo = dataset_wapo.docs_store()

In [19]:
docstore_nyt.get("532").headline

'DEATH TOLL AT 95 IN SAN JUAN BLAZE AS SEARCH GOES ON'

In [29]:
docstore_wapo.get("4b0d69ae-d85a-11e2-a9f2-42ee3912ae0e")

WapoDoc(doc_id='4b0d69ae-d85a-11e2-a9f2-42ee3912ae0e', url='https://www.washingtonpost.com/politics/2013/07/14/4b0d69ae-d85a-11e2-a9f2-42ee3912ae0e_story.html', title='Reid-McConnell clash latest evidence genteel Senate has turned into a fight club', author='Paul Kane', published_date=1373846443000, kicker='Politics', body='The 2014 midterm elections are just around the corner. These are the races to watch.\nThe U.S. Senate, once considered the most exclusive and chummiest club in America, has in recent years been transformed into an ideological war zone, where comity and compromise have lost their allure, while confrontation and showmanship now pay big dividends.\n\n\nReid’s move\nSuddenly, the world’s greatest deliberative body just isn’t that much fun anymore.\nA three-minute mashup of a spat between Senate Majority Leader Harry Reid and Minority Leader Mitch McConnell on the Senate floor.\n“I miss it like an abscessed tooth,” former senator Christopher S. Bond (R-Mo.), a 24-year ve

In [ ]:
#create indices

